In [1]:
import syft as sy

In [2]:
domain = sy.Domain(name="Bob")

In [3]:
root_client = domain.get_root_client()

In [4]:
client = domain.get_client()

In [5]:
# code_path = "/Users/madhavajay/dev/PySyft/packages/syft/scripts/code.py"
# code_str = ""
# with open(code_path) as f:
#     code_str = f.read()

# print("RAW CODE")
# print("=============================")
# print(code_str)
# print("=============================")
# print("\n\n")

In [6]:
code_str = """
## USER CODE
print("test")
import syft

class Test:
    def hello(self) -> str:
        return "syckle hello"
    

## Sandboxing Code
module_type = type(syft)

if "sandbox" not in syft.__dict__:
    syft.__dict__["sandbox"] = module_type(name="sandbox")

parent = syft.__dict__["sandbox"]

namespace = "mylib"

import_path = f"{namespace}.Test"
module_parts = import_path.split(".")

klass = module_parts.pop()
Test.__module__ = f"syft.sandbox.{namespace}"
Test.__name__ = klass

for n in module_parts:
    if n not in parent.__dict__:
        parent.__dict__[n] = module_type(name=n)
    parent = parent.__dict__[n]

parent.__dict__[Test.__name__] = Test

print("done", dir(syft.sandbox))
"""

In [7]:
import ast
tree = ast.parse(code_str)
root_client.secure_exec(tree)

Remote Secure Exec
test
done ['__doc__', '__loader__', '__name__', '__package__', '__spec__', 'mylib']
Accepting, code executed successfully!


In [8]:
try:
    client.secure_exec(tree)
except Exception as e:
    print("Cant because not root")

[2021-06-16T16:27:59.451775+1000][CRITICAL][logger]][14431] You are not Authorized to access this service


Cant because not root


In [9]:
import sys

In [10]:
sys.modules["syft"].sandbox

<module 'sandbox'>

In [11]:
sy.sandbox.mylib.Test

syft.sandbox.mylib.Test

In [12]:
from syft.core.node.common.action.function_or_constructor_action import (
    RunFunctionOrConstructorAction,
)
from syft.core.common.uid import UID
from syft.core.io.address import Address
from syft.core.node.common.action.run_class_method_action import RunClassMethodAction
from syft.core.pointer.pointer import Pointer

In [13]:
test = RunFunctionOrConstructorAction(
    path="syft.sandbox.mylib.Test",
    args=tuple(),
    kwargs={},
    id_at_location=UID(),
    address=root_client.address,
    msg_id=UID(),
)

In [14]:
test

RunClassMethodAction Test(, )

In [15]:
fake_pointer = Pointer(id_at_location=test.id_at_location, client=root_client)
fake_pointer.path_and_name=test.path

In [16]:
fake_pointer.id_at_location

<UID: f5770b1ecf974c53ac062ef118cd6f2d>

In [17]:
root_client.send_immediate_msg_without_reply(msg=test)

In [18]:
root_client.store.pandas

,ID,Tags,Description,object_type
0,<UID: f5770b1ecf974c53ac062ef118cd6f2d>,[],,<class 'syft.sandbox.mylib.Test'>


In [19]:
hello = RunClassMethodAction(
    path="syft.sandbox.mylib.Test.hello",
    _self=fake_pointer,
    args=[],
    kwargs={},
    id_at_location=UID(),
    address=root_client.address,
    msg_id=UID(),
)

In [20]:
root_client.send_immediate_msg_without_reply(msg=hello)

[2021-06-16T16:27:59.521374+1000][CRITICAL][logger]][14431] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs for the requested field.
[2021-06-16T16:27:59.522070+1000][CRITICAL][logger]][14431] 'Class' object has no attribute 'Pointer'
[2021-06-16T16:27:59.522514+1000][CRITICAL][logger]][14431] '__getattr__ failed, Pointer is not present on the object, nor the AST attributes!'


KeyError: '__getattr__ failed, Pointer is not present on the object, nor the AST attributes!'

In [ ]:
hello_ptr = Pointer(id_at_location=hello.id_at_location, client=root_client)

In [ ]:
import torch as th
t = th.Tensor([1, 2, 3])

In [ ]:
tensor_pointer = t.send(root_client)

In [ ]:
tensor_pointer.path_and_name

In [ ]:
a2 = RunClassMethodAction(
    path="torch.Tensor.add",
    _self=tensor_pointer,
    args=[tensor_pointer],
    kwargs={},
    id_at_location=UID(),
    address=root_client.address,
    msg_id=UID(),
)

In [ ]:
root_client.send_immediate_msg_without_reply(msg=a2)

In [ ]:
root_client.store.pandas